<a href="https://colab.research.google.com/github/Angel353/CADII_Equipo1/blob/main/Airbnb_Bases/CDMX/Regresi%C3%B3n_No_Lineal/Dashboard_No_Lineal_CDMXipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

In [ ]:
#Instalamos la libreria de PLOTLY
%pip install plotly

In [ ]:
#Instalamos librería para análisis de variables categóricas
%pip install funpymodeling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.6/204.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.

In [ ]:

#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
!npm install localtunnel


up to date, audited 23 packages in 540ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
%pip install streamlit-option-menu

In [ ]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from streamlit_option_menu import option_menu
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración de la página con iconos y layout
st.set_page_config(
    page_title="Airbnb CDMX Dashboard",
    page_icon="🏠",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Cargar los datos
@st.cache_data
def load_data():
    data = pd.read_csv('Mexico_Outliers_y_Categorización.csv', encoding='utf-8', index_col=0)
    return data

# Preprocesar datos
def preprocess_data(data):
    data_cuant = data.copy()
    data_cuant.drop(["latitude", "longitude", "availability_30", "last_scraped", "source", "host_name",
                     "host_verifications", "neighbourhood_cleansed", "property_type", "room_type", "bathrooms_text",
                     "amenities", "calendar_last_scraped", "bathroom_text_clean", "neighbourhood_Category",
                     "amenitiesCat", "hostTotalListingsCat", "accommodatesCat", "minimumNightsCategory",
                     "maximumNightsCategory", "availabilityYearCat", "hostYearsCat", "hostResponseCat",
                     "hostAcceptanceCat", "hostVerifCat", "bedsCategory", "reviewsMonthCat", "reviewScoresCat",
                     "reviewLocationCat", "reviewRatingCat", "reviewCleanCat", "reviewCommunicationCat"], axis=1, inplace=True)

    data_cuant = data_cuant.select_dtypes(include=[np.number])
    return data_cuant

# Funciones de regresión no lineal
def cuadratic(x, a, b, c):
    return a * x**2 + b * x + c

def exponential(x, a, b, c):
    return a * np.exp(-b * x) + c

def inverse_polynomial(x, a, b, c):
    return a / b * x**2 + c * x

def absolute_value(x, a, b, c):
    return abs(a * x**2 + b * x + c)

# Función de graficar heatmap
def plot_heatmap(data):
    corr_matrix = data.corr()
    fig = px.imshow(corr_matrix, text_auto=True, color_continuous_scale='RdBu_r', title="Heatmap de Correlaciones")
    return fig

# Función para mostrar boxplot
def plot_boxplot(data, y_var):
    fig = px.box(data, y=y_var, title=f"Boxplot de {y_var}", color_discrete_sequence=['#636EFA'])
    return fig

# Función para la regresión no lineal y gráfica
def plot_regression(data, x_var, y_var, func):
    x = data[x_var]
    y = data[y_var]
    try:
        popt, _ = curve_fit(func, x, y)
        x_line = np.linspace(x.min(), x.max(), 100)
        y_line = func(x_line, *popt)

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Datos', marker=dict(color='rgba(0, 150, 255, .8)')))
        fig.add_trace(go.Scatter(x=x_line, y=y_line, mode='lines', name='Regresión', line=dict(color='rgba(255, 0, 0, .8)', width=2)))

        r2 = r2_score(y, func(x, *popt))
        R = np.sqrt(r2)  # Coeficiente de correlación
        return fig, r2, R
    except:
        st.error(f"No se pudo ajustar la regresión para {y_var} vs {x_var}")
        return None, None, None

# Dashboard principal
def main():
    st.markdown("<h1 style='text-align: center; color: #00BFFF;'>📊 Análisis Airbnb CDMX 📊</h1>", unsafe_allow_html=True)

    # Cargar y preprocesar datos
    data = load_data()
    data_cuant = preprocess_data(data)

    # Sidebar para navegación
    with st.sidebar:
        selected = option_menu("Menú Principal", ["Resumen", "Correlaciones", "Regresión No Lineal", "Visualizaciones Adicionales"],
                               icons=['house', 'bar-chart-line', 'activity', 'palette'], menu_icon="cast", default_index=0)

    st.markdown("<p style='text-align: center;'>Sebastian Contreras</p>", unsafe_allow_html=True)

    if selected == "Resumen":
        st.subheader("📈 Resumen de Datos Numéricos")
        st.dataframe(data_cuant.describe())

        st.subheader("📉 Distribución de Precios")
        fig = px.histogram(data_cuant, x='price', nbins=50, title="Distribución de Precios", color_discrete_sequence=['#EF553B'])
        st.plotly_chart(fig, use_container_width=True)

    elif selected == "Correlaciones":
        st.subheader("🔗 Mapa de Correlaciones")
        if st.checkbox("Mostrar Heatmap de Correlaciones"):
            fig = plot_heatmap(data_cuant)
            st.plotly_chart(fig, use_container_width=True)

        # Mostrar tabla de correlaciones
        st.subheader("📊 Tabla de Correlaciones")
        corr_table = data_cuant.corr()
        st.dataframe(corr_table)

    elif selected == "Regresión No Lineal":
        st.subheader("📉 Análisis de Regresión No Lineal")

        # Selectores
        x_var = st.selectbox("Seleccione Variable X", ['host_response_time_num', 'instant_bookable', 'calculated_host_listings_count', 'beds', 'number_of_reviews_ltm', 'bathrooms'])
        y_var = st.selectbox("Seleccione Variable Y", ['host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'accommodates', 'reviews_per_month', 'price'])
        formula = st.selectbox("Seleccione el tipo de regresión", ['Cuadrática', 'Exponencial', 'Polinomial Inversa', 'Valor Absoluto'])

        # Mapeo de la función seleccionada
        funciones_regresion = {
            "Cuadrática": cuadratic,
            "Exponencial": exponential,
            "Polinomial Inversa": inverse_polynomial,
            "Valor Absoluto": absolute_value
        }

        # Mostrar gráfico de regresión
        if st.button("Generar Regresión"):
            fig, r2, R = plot_regression(data_cuant, x_var, y_var, funciones_regresion[formula])
            if fig:
                st.plotly_chart(fig, use_container_width=True)
                st.metric("Coeficiente de Determinación (R²)", f"{r2:.4f}")
                st.metric("Coeficiente de Correlación (R)", f"{R:.4f}")

    elif selected == "Visualizaciones Adicionales":
        st.subheader("🎨 Visualizaciones Adicionales")

        # Visualización de boxplot
        col1, col2 = st.columns(2)
        with col1:
            y_box_var = st.selectbox("Seleccione Variable para Boxplot", data_cuant.columns)
            fig_box = plot_boxplot(data_cuant, y_box_var)
            st.plotly_chart(fig_box, use_container_width=True)

        # Visualización de scatterplot matrix
        with col2:
            selected_vars = st.multiselect("Seleccione Variables para Matriz de Dispersión", data_cuant.columns)
            if selected_vars:
                st.subheader("📊 Matriz de Dispersión")
                pairplot_fig = sns.pairplot(data_cuant[selected_vars])
                st.pyplot(pairplot_fig)

        # Visualización de histogramas con curvas de densidad
        st.subheader("📈 Histogramas con Curvas de Densidad")
        selected_hist_vars = st.multiselect("Seleccione Variables para Histogramas", data_cuant.columns)
        for hist_var in selected_hist_vars:
            fig_hist = px.histogram(data_cuant, x=hist_var, histnorm='probability density', title=f"Histograma de {hist_var}")
            st.plotly_chart(fig_hist, use_container_width=True)

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.127.103.136



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.127.103.136:8501

your url is: https://thirty-geckos-chew.loca.lt
/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.

/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1119: FutureWarning:

use_inf_as_na option is deprecated and will b